## **Analyse des données avant merge des bases**

In [2]:
import pandas as pd
import numpy as np 
from helpers2 import S3Connection
import helpers2 
import matplotlib.pyplot as plt

In [3]:
s3 = S3Connection(bucket_name="clichere/diffusion")

Connection successful


In [4]:
path1 = "DPE/DPE_après_2021.parquet"
DPE = s3.get_tables_from_s3(path1)

In [5]:
path2 = "Valeursfoncières/vf.parquet"
vf = s3.get_tables_from_s3(path2)

### **premières statistiques descriptives**

In [6]:
DPE.columns

Index(['Date_établissement_DPE', 'Modèle_DPE', 'Version_DPE', 'Etiquette_DPE',
       'Etiquette_GES', 'Année_construction', 'Type_bâtiment',
       'Période_construction', 'Surface_habitable_logement',
       'Code_INSEE_(BAN)', 'Adresse_(BAN)', 'Score_BAN',
       'Coordonnée_cartographique_X_(BAN)',
       'Coordonnée_cartographique_Y_(BAN)', 'N°_étage_appartement',
       'N°_région_(BAN)'],
      dtype='object')

In [7]:
print("Nombre de lignes par colonnes:", len(DPE))
total_nan = DPE.isna().sum().sum()
print(f"Nombre total de NaN : {total_nan}")
nan_par_colonne = DPE.isna().sum()
top_10_nan_colonnes = nan_par_colonne.sort_values(ascending=False).head(10)
print("Les 10 colonnes avec le plus de NaN :")
print(top_10_nan_colonnes)

Nombre de lignes par colonnes: 4074868
Nombre total de NaN : 2570649
Les 10 colonnes avec le plus de NaN :
Année_construction                   1257443
N°_étage_appartement                 1163134
Surface_habitable_logement             44799
Adresse_(BAN)                          22236
N°_région_(BAN)                        18877
Coordonnée_cartographique_Y_(BAN)      16040
Code_INSEE_(BAN)                       16040
Score_BAN                              16040
Coordonnée_cartographique_X_(BAN)      16040
Période_construction                       0
dtype: int64
